In [8]:
import os
import pandas as pd
#from binance.client import Client
import math
#from pyalgotrading.utils.candlesticks import heikinashi
from IPython.display import clear_output
import time
import sys
import datetime

In [ ]:
class TradeBot:
    def __init__(self,user_key,secret_key,symbol):
        self.user_key=user_key
        self.secret_key=secret_key
        self.binance_client=Client(user_key, secret_key)
        self.symbol=symbol

    def UP_BUY(self,Quantity):
        self.binance_client.create_order(
        symbol=self.symbol,
        type='MARKET',
        side='BUY',
        quantity=Quantity
        )
    def UP_SELL(self,Quantity):
        self.binance_client.create_order(
        symbol=self.symbol,
        type='MARKET',
        side='SELL',
        quantity=Quantity
        )
    def CurrentCoinPrice(self):
        return float(self.binance_client.get_symbol_ticker(symbol=self.symbol)['price'])
        
    def Quantity(self):
        account_balance=float(self.binance_client.get_asset_balance(asset='USDT')['free'])
        coin_price=float(self.binance_client.get_symbol_ticker(symbol=self.symbol)['price'])
        quantity_to_be_purchased=math.floor(account_balance/coin_price)
        
        return quantity_to_be_purchased,coin_price

    def compute_HA(self,interval):
        prev_date=datetime.datetime.utcnow().date()-datetime.timedelta(days=1)
        df=pd.DataFrame(self.binance_client.get_historical_klines(self.symbol, interval,str(prev_date)))
        df = df.iloc[:, :6]
        # ascribe names to columns
        df.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']
        # convert timestamp to date format and ensure ohlcv are all numeric
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        for col in df.columns[1:]:
            df[col] = pd.to_numeric(df[col])
        df_ha=heikinashi.HeikinAshi(japanese_data=df)
        return df_ha
    def isUP(self,interval='1h'):
        df=self.compute_HA(interval)
        open_price=df.iloc[-1]['open']
        close_price=df.iloc[-1]['close']
        if close_price>=open_price:
            return True
        return False
    
    def CandleStickMean(self,interval='15m'):
        df=self.compute_HA(interval)
        df['CandleLength']=df['close']-df['open']
        positive=df[df['CandleLength']>=0]
        negative=df[df['CandleLength']<0]
        pos=positive['CandleLength'].mean()
        neg=-negative['CandleLength'].mean()
        return pos,neg
    
    def CurrentCandleStick(self,interval='15m'):
        df=self.compute_HA(interval)
        open_price=df.iloc[-1]['open']
        close_price=df.iloc[-1]['close']
        difference=close_price-open_price
        if difference<0:
            difference=-difference
        return difference
    
    def DownOrUp(self,interval='15m'):
        df=self.compute_HA(interval)
        open_price=df.iloc[-1]['open']
        close_price=df.iloc[-1]['close']
        return open_price ,close_price
    
    def ShiekhBot(self):
        while(1):
            open_price,close_price=self.DownOrUp()
            if self.isUP('15m'):
                pos,neg=self.CandleStickMean()
                pos=pos/5
                if self.CurrentCandleStick()>pos:
                    order_quantity,coin_price=self.Quantity()
                    self.UP_BUY(order_quantity)
                    while(1):
                        if self.isUP('15m'):
                            print("Bought")
                            clear_output(wait=True)
                        else:
                            print("Sold")
                            clear_output(wait=True)
                else:
                    print("Consolidating")
                    clear_output(wait=True)
            else:
                print("DownState")
                clear_output(wait=True)
                

    def RunBot(self):
        while(1):
            open_price,close_price=self.DownOrUp()
            if ((open_price<close_price) & ((self.isUP()==self.isUP('15m')) & (self.isUP()==self.isUP('1m'))) ):
                pos,neg=self.CandleStickMean()
                pos1m,neg1m=self.CandleStickMean('1m')
                
                if (self.CurrentCandleStick()>pos) &(self.CurrentCandleStick('1m')>pos1m):
                    order_quantity,coin_price=self.Quantity()
                    self.UP_BUY(order_quantity)
                    while(1):
                        if ((open_price<close_price) & ((self.isUP()==self.isUP('15m')) & (self.isUP()==self.isUP('1m'))) ) :
                            currentcoinprice=self.CurrentCoinPrice()
                            if currentcoinprice>=coin_price:
                                profit=(currentcoinprice*order_quantity)-(coin_price*order_quantity)
                                print('Profit = ',profit)
                            else:
                                loss=(currentcoinprice*order_quantity)-(coin_price*order_quantity)
                                print('Loss = ',loss)
                            clear_output(wait=True)
                        else:
                            self.UP_SELL(order_quantity)
                            break
                else:
                    print('Consolidating')
                    clear_output(wait=True)
            elif ((open_price>close_price) & ((self.isUP()==self.isUP('15m')) & (self.isUP()==self.isUP('1m')))):
                pos,neg=self.CandleStickMean()
                pos1m,neg1m=self.CandleStickMean('1m')
                
                if (self.CurrentCandleStick()>neg) & (self.CurrentCandleStick('1m')>neg1m):
                    order_quantity,coin_price=self.Quantity()
                    self.DOWN_BUY(order_quantity)
                    while(1):
                        if ((open_price>close_price) & ((self.isUP()==self.isUP('15m')) & (self.isUP()==self.isUP('1m')))):
                            currentcoinprice=self.CurrentCoinPrice()
                            if currentcoinprice<=coin_price:
                                profit=(coin_price*order_quantity)-(currentcoinprice*order_quantity)
                                print('Profit = ',profit)
                            else:
                                loss=(coin_price*order_quantity)-(currentcoinprice*order_quantity)
                                print('Loss = ',loss)
                            clear_output(wait=True)
                        else:
                            self.DOWN_SELL(order_quantity)
                            break
                else:
                    print('Consolidating')
                    clear_output(wait=True)
                    
            else:
                print('Consolidating')
                clear_output(wait=True)
                



In [ ]:
symbol='BETAUSDT'
user_key= 1# Enter you user key 

secret_key=1 #Enter Secret Key

In [ ]:
TB=TradeBot(user_key,secret_key,symbol)

In [ ]:
TB.ShiekhBot()

Consolidating


In [ ]:
TB.Quantity()

(0, 3.8696)